In [1]:
import sys
import os
import numpy as np
from _3dpu_using_dfs import *
ROOT = "../"

In [2]:
import yaml

# Load the YAML file
with open(ROOT + 'paths.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access paths
data_path = config['paths']["data_big"]




In [4]:
import nibabel as nb
t = 3
data = nb.load(data_path).get_fdata()
data = np.array(data)[:,:,:,t]

In [34]:
def iterative_dfs(C, start):
    stack = [(start, [start])]
    C.visited[start] = True
    path_dic = {start: 0}
    while stack:
        v, path = stack.pop()
        C.visited[v] = True
        path_dic[v] = len(path) - 1
        print(stack)
        for neighbour in C.Res_graph[v]:
            if not C.visited[neighbour]:
                newPath = path + [neighbour]
                stack.append((neighbour, newPath))
            elif neighbour in path:
                cycle_start_index = path_dic[neighbour]
                cycle = path[cycle_start_index:] + [neighbour]

                C.cycles.append(cycle)
                for point in cycle:
                    C.visited[point] = True
                    C.incycles[point] = True
                return cycle

    return stack

def detect_cycles_iterative(C):
    for i in tqdm(range(len(C.mapping))):
        if not C.visited[i]:
            iterative_dfs(C, i)
                

In [23]:
C.cycles[0]

[6378,
 624569,
 1345162,
 624608,
 6451,
 617799,
 171,
 612685,
 1334048,
 612684,
 612656,
 135,
 6378]

In [5]:
C = Resiuals(data)
C.map_nodes()
C.create_graph()
C.cycles = []
C.incycles = [1]*len(C.mapping)
C.connex =[1]*len(C.mapping)
C.visited = [False]*len(C.mapping)
# iterative_dfs(C,6378)

100%|██████████| 2023266/2023266 [00:02<00:00, 951128.58it/s] 


In [90]:
C.Res_graph = dict()
for res in C.list_res:
    C.Res_graph[C.res_ordre[res]] = []
for res in tqdm(C.list_res):
    C.nodes_of_not_boundary(res)

100%|██████████| 2023266/2023266 [00:11<00:00, 183695.36it/s]


In [99]:
C.visited = set()
C.open_paths = []
def optimized_dfs(C):
    
    for start_node in tqdm(C.starting_open_paths):
        if start_node not in C.visited:
            C.visited.add(start_node)
            node = start_node
            path = []
            while not C.end_of_path[node]:
                path.append(node)
                next_nodes = [n for n in C.Res_graph[node] if not C.incycles[n]]
                node = next_nodes[0]
            
            C.open_paths.append(path)

                

    return C.open_paths

def fill_open_paths(C,paths,layer):
    antecedant = dict()
    visited = set()
    K = []
    for point in tqdm(layer):
        paths[point] = [point]
        antecedant[point] = point
    cpt = 0
    while layer != {-1}:
        # cpt += 1
        # if cpt % 100 == 0:
        #     print(len(layer))
        new_layer = set()
        for point in (layer):
            if point != -1 :
                next_nodes = [n for n in C.Res_graph[point] if (n not in visited)]
                if next_nodes:
                    
                    next_node = next_nodes[0]
                    assert next_node not in visited, "Error we have already visited said point"
                    antecedant[next_node] = antecedant[point]
                    new_layer.add(next_node)
                    paths[antecedant[point]].append(next_node)
                    visited.add(next_node)
                    K.append(next_node)

                else:
                    new_layer.add(-1)

        assert layer != new_layer, ("there is a repetition",layer,new_layer)
        layer = new_layer
        

    C.open_paths = list(paths.values())
    return K,antecedant
    


In [ ]:

def fill_open_paths(C,paths,layer):
    antecedant = dict()
    visited = set()
    set_paths = dict()
    index_paths = dict()
    cycles = []
    K = []
    for point in tqdm(layer):
        paths[point] = [point]
        antecedant[point] = point
        index_paths[point] = dict()
        set_paths[point] = dict()

    while layer != {-1}:

        new_layer = set()
        for point in (layer):
            
            if point != -1 :
                next_nodes = [n for n in C.Res_graph[point] if (not C.incycles[n] and n not in visited)]
                if next_nodes == []:
                    new_layer.add(-1)
                else:
                    next_node = next_nodes[0]
                    visited.add(next_node)

                    if next_node in set_paths[antecedant[point]]:
                        ind = index_paths[antecedant[point]][next_node]
                        cycle = paths[ind:].append(next_node)
                        cycles.append(cycle)
                        point = paths[ind-1]
                    
                    else :
                        next_nodes = [n for n in C.Res_graph[point] if (not C.incycles[n] and n not in visited)]
                        paths[antecedant[point]].append(next_node)
                        set_paths[antecedant[point]].add(next_node)
                        index_paths[antecedant[point]][next_node] = len(paths[antecedant[point]]) - 1
                        new_layer.add(next_node)
                        antecedant[next_node] = antecedant[point]


        assert layer != new_layer, ("there is a repetition",layer,new_layer)
        layer = new_layer
        

    C.open_paths = list(paths.values())
    C.cycles = cycles
    return K,antecedant
    

In [93]:
C.incycles = [False]*len(C.mapping)
C.open_paths = []
C.starting_open_paths = []
C.cycles = []
C.fill_starting_open_paths()
for path in C.cycles:
    for point in path:
        C.incycles[point] = True


  0%|          | 0/2023266 [00:00<?, ?it/s]

100%|██████████| 2023266/2023266 [00:01<00:00, 1583281.53it/s]


In [94]:
C.open_paths=[]
paths = dict()
layer = set(C.starting_open_paths)
K,antecdant = fill_open_paths(C,paths,layer)

100%|██████████| 16578/16578 [00:00<00:00, 635865.57it/s]


In [95]:
Points = set()
for open_path in tqdm(C.open_paths):
    Points.update(open_path)

100%|██████████| 16578/16578 [00:00<00:00, 60485.51it/s]


In [96]:
Dict = dict()
Set_paths = [set(open_path) for open_path in C.open_paths]
for point in tqdm(Points):
    Dict[point] = []
for path in tqdm(Set_paths):
    for point in path:
        Dict[point].append(path)

100%|██████████| 16578/16578 [00:00<00:00, 20815.11it/s]


In [98]:
for point in tqdm(Points):
    if len(Dict[point]) > 1 :
        print(point,Dict[point])

100%|██████████| 1548332/1548332 [00:00<00:00, 2675139.95it/s]


In [82]:
K.index(671798)

176023

In [51]:
Dict[671798]

[{43196,
  43281,
  49426,
  49449,
  49450,
  49505,
  55503,
  55571,
  61680,
  67715,
  658099,
  658130,
  658158,
  671798,
  671825,
  671826,
  671887,
  678714,
  692258,
  692286,
  1377504,
  1377505,
  1384085,
  1397189},
 {49427, 664876, 671798, 1384055}]

In [17]:
Closed_points = set()
Open_points = set()

for path in C.open_paths:
    for point in path:
            C.incycles[point] = True
            Open_points.add(point)

for path in C.cycles:
    for point in path:
            C.incycles[point] = True
            Closed_points.add(point)

In [18]:
Not_in_cycles = []
for node in range(len(C.mapping)):
    if not C.incycles[node] :
        Not_in_cycles.append(node)
len(Not_in_cycles)

203616

In [19]:
True in C.end_of_path

True

In [22]:
node = 10
boolean = C.Res_graph[10][0] in Not_in_cycles
while  boolean :
    print(node)
    Not_in_cycles.remove(node)
    boolean = False
    for k in C.Res_graph[node]:
        if k in Not_in_cycles:
            boolean = True
            node = k



10
612531
1333919
1333920
1333921
37
6256
1345003
6257
38
612560
1333944
612585
90
1338584
617711
1338617
118
1333980
612608
1333962
1333961
612607
612638
612665
1334029
612664
612637
117
1338614
617709
617676
6288
624434
6254
617612


In [24]:
C.Res_graph[617612]

[10, 1338500]

In [104]:
C.Res_graph[10] in Not_in_cycles

False

In [83]:
for op in C.open_paths:
    if 617565 in op:
        print(op)

[612499, 1333876, 0, 617565]


In [92]:
actual_starting_open_paths = []
actual_ending_points = []
for op in C.open_paths:
    actual_starting_open_paths.append(op[0])
    actual_ending_points.append(op[-1])

In [89]:
starting_open_paths_left = []
ending_pointsè
for point in C.starting_open_paths:
    if point not in actual_starting_open_paths:
        starting_open_paths_left.append(point)



In [1]:
len(C.starting_open_paths) - len(actual_starting_open_paths) 

NameError: name 'C' is not defined

In [94]:
for op in C.cycles:
    if 1933316 in op:
        print(op)

1933316


AttributeError: 'Resiuals' object has no attribute 'end_of_path'

In [35]:
for op in C.open_paths:
    if 1248021 in op:
        print(op)

[1933316, 1241083, 529714, 1248021, 529743, 523860, 1234296, 1926622]


In [32]:
C.end_of_path[1926622]

True

In [ ]:
for k in tqdm(range(C.list_res)):

In [ ]:
for k in Begin :
    if k not in C.visited:
        print(k)

In [ ]:
my_set = {1, 2, 3, 4, 5, 6}
my_list = [2, 4, 9]

my_set.difference_update(my_list)
print(my_set)

{1, 3, 5, 6}


In [ ]:
len(Begin)

16578

In [ ]:
len(C.linear_paths)

10727

In [ ]:
set(C.dict_open_paths.values())

{0, 1}

In [ ]:
for op in C.open_paths:
    if 58329 in op:
        print(op)

[58329, 126712, 193867]
[58329, 126712, 193867]
[58329, 126712, 193867]


In [ ]:
len(C.open_paths)/3

38.0

In [ ]:
C.starting_open_paths[123]

63433

In [ ]:
C.Res_graph[63433]

[63424, 4891]

In [ ]:
for adjacency in C.Res_graph.values():
    if 4891 in adjacency :
        print(adjacency)

[63424, 4891]
[63424, 4891]


In [43]:
Poin
(list(paths.values()))

[[1933316, 1241046, 1933276],
 [1998855,
  583442,
  1992222,
  1301849,
  1301820,
  577199,
  1985568,
  577198,
  1301789,
  583351,
  1998765],
 [1343496,
  4737,
  616341,
  616311,
  4671,
  10916,
  1349914,
  10915,
  1343430,
  1343429,
  4670,
  616276,
  4639,
  1343391,
  4640,
  1337341,
  616250,
  1337314,
  4606,
  10858,
  1349848,
  629410,
  629448,
  17007,
  636174,
  636138,
  1356254,
  23069,
  1362749,
  642909,
  1362785,
  23095,
  636173,
  1356313,
  636172,
  1356280,
  636137,
  23068,
  1362747,
  642908,
  1362784,
  1362783,
  23094,
  16965,
  629407,
  629373,
  1349814,
  16908,
  1356223,
  636080,
  16871,
  629318,
  10763,
  622526,
  10731,
  629285,
  16839,
  1356157,
  16838,
  1349752,
  1349753,
  629319,
  629343,
  1349815,
  629374,
  1349845,
  10855,
  4604,
  1337310,
  1337311,
  616218,
  4576,
  622599,
  1343284,
  10793,
  1349781,
  629344,
  10821,
  622630,
  10857,
  1349847,
  629409,
  629447,
  10913,
  1343427,
  4669,
 

In [ ]:
C.incycles[129011]

False

In [ ]:
J = [len(cycle) for cycle in C.cycles]

In [ ]:
max(J)

89

In [ ]:
C.cycles = []
C.incycles = [1]*len(C.mapping)
C.visited = [1]*len(C.mapping)
C.dfs_iterative(613037)

In [ ]:
cpt

58

In [ ]:
for cycle in C.cycles:
    if 1334354 in cycle:
        print(len(cycle))

7416


In [ ]:
for cycle in C.cycles:
    if 613037 in cycle:
        print(cycle)

[613018, 613037, 1334354, 634, 6888, 1345645, 1345646, 613063, 1334377, 1334378, 613064, 1334321, 613017, 1334340, 613018]
[613037, 1334354, 634, 6888, 1345645, 1345646, 613063, 1334377, 1334378, 613064, 1334321, 613017, 1334340, 613037]


In [ ]:
C.visited

[]

In [ ]:
len(np.where(np.array(C.incycles) == 1)[0])

606743

In [ ]:
for open_path in C.open_paths:
    assert C.is_boundary(open_path[0]), "you hav error"

In [ ]:
L = [len(C.Res_graph[node]) for node in C.starting_open_paths]
L

[1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,


In [ ]:
incycles = [1]*len(C.mapping)
def detect_cycles(C):
    path_indices = {}
    for v in tqdm(range((len(C.mapping)))):
        if C.connex[v] != -1:
            dfs_iterative(C,v,path_indices)

def dfs_iterative(C, start,path_indices):
    stack = [(start, -1)]
    path_indices[start] = 0
    path = [start]
    while stack:
        node, parent = stack.pop()
        if C.connex[node] != -1:
            for neighbour in C.Res_graph[node]:
                if neighbour != parent:
                    if neighbour in path:
                        cycle_start_index = path.index(neighbour)
                        C.cycles.append(path[cycle_start_index:] + [neighbour])
                        for point in path[cycle_start_index:] + [neighbour]:
                            C.connex[point] = -1
                            C.incycles[point] = -1
                    else:
                        stack.append((neighbour, node))
                        path = path + [neighbour]
                        path_indices[neighbour] = len(path)
                    

            C.connex[node] = -1



In [ ]:
C.cycles = []
for point in range(len(C.mapping)):
    C.connex[point] = 2

detect_cycles(C)

100%|██████████| 2023266/2023266 [00:31<00:00, 64708.27it/s]  


In [ ]:
len(np.where(np.array(incycles) == 1)[0])/len(C.mapping)

0.3310459425503122

In [ ]:
669794/len(C.mapping)

0.3310459425503122

In [ ]:
L = [len(compo) for compo in C.connected_components.values()]
bad = L.index(1)
colour = list(C.connected_components.keys())[bad]
C.connected_components[colour]

[28]

In [ ]:
C.Res_graph[28]

[1338522, 1338523]

In [ ]:
from tqdm import tqdm
for r in tqdm(C.list_res):
    C.Res_graph[R[r]] = []

  0%|          | 0/6069798 [00:00<?, ?it/s]

100%|██████████| 6069798/6069798 [00:09<00:00, 632648.60it/s] 


In [ ]:
R

[0]

In [ ]:
for res in tqdm(C.list_res):
    i,j,k,axe,value = res
    # if i not in [0,X-1] and j not in [0,Y-1] and k not in [0,Z-1]:
    C.nodes_of_not_boundary(res)

  0%|          | 0/2023266 [00:00<?, ?it/s]


KeyError: 0

In [ ]:
C.identify_connected_component()

In [ ]:
D = (C.connected_components).values()
Number = [len(k) for k in D]
R = sorted(Number)
R

[0]

In [ ]:
R

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
Number

[2,
 1781553,
 3,
 5,
 6,
 2,
 2,
 2,
 5,
 2,
 2,
 5,
 1,
 5,
 5,
 1,
 4,
 2,
 8,
 4,
 2,
 6,
 2,
 4,
 10,
 4,
 4,
 4,
 4,
 16,
 6,
 4,
 6,
 15,
 8,
 4,
 4,
 8,
 4,
 4,
 12,
 4,
 2,
 3,
 6,
 24,
 4,
 12,
 6,
 4,
 4,
 2,
 4,
 4,
 6,
 4,
 4,
 8,
 8,
 22,
 4,
 14,
 8,
 6,
 2,
 4,
 6,
 2,
 10,
 4,
 4,
 4,
 2,
 7,
 4,
 8,
 10,
 14,
 4,
 4,
 6,
 6,
 6,
 4,
 9,
 4,
 3,
 8,
 4,
 2,
 12,
 6,
 2,
 2,
 4,
 6,
 10,
 4,
 4,
 9,
 6,
 4,
 8,
 4,
 4,
 10,
 6,
 6,
 6,
 4,
 4,
 6,
 6,
 1,
 6,
 4,
 2,
 10,
 4,
 6,
 4,
 4,
 4,
 6,
 4,
 4,
 14,
 6,
 4,
 1,
 4,
 4,
 6,
 3,
 4,
 4,
 6,
 4,
 4,
 6,
 10,
 5,
 4,
 4,
 14,
 4,
 4,
 10,
 6,
 12,
 8,
 4,
 16,
 2,
 8,
 3,
 6,
 4,
 4,
 4,
 4,
 14,
 12,
 6,
 4,
 4,
 4,
 4,
 4,
 10,
 9,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 4,
 4,
 10,
 4,
 4,
 4,
 5,
 6,
 10,
 6,
 5,
 4,
 6,
 6,
 4,
 4,
 8,
 11,
 4,
 11,
 2,
 4,
 14,
 3,
 4,
 2,
 3,
 1,
 2,
 1,
 1,
 3,
 5,
 8,
 4,
 3,
 5,
 2,
 3,
 4,
 7,
 2,
 2,
 23,
 2,
 3,
 7,
 2,
 2,
 16,
 3,
 3,
 3,
 4,
 2,
 2,
 6,
 2,
 19,
 3

In [ ]:
0 in (set(list(C.connex.values())))

False

In [ ]:
R = []
for key in tqdm(C.Res_graph.keys()):
    if len(C.Res_graph[key]) not in [0,1,2,3]:
        R.append(key)

100%|██████████| 2023266/2023266 [00:00<00:00, 2402570.20it/s]
